## RAG-application
### Setup och Ladda Databas

In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv

# Vektordatabas och Embeddings
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

# Prompting, LLM och för att extrahera text från LLM-output
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

# ==========================================================
# 1. Ladda ENV och definiera sökvägar
# ==========================================================
load_dotenv()

BASE_DIR = Path(r"C:\Users\Dator\Documents\Data_Science\11_Examensarbete\green_power_sweden")
DB_DIR = BASE_DIR / "data" / "03_vector_db" / "green_power_sweden_db" 

# ==========================================================
# 2. Ladda embeddingmodell
# ==========================================================
print("Laddar embedding-modell...")

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': False}
)

# ==========================================================
# 3. Ladda vektordatabasen
# ==========================================================
print(f"Laddar databas från {DB_DIR}...")

# Försök ladda databasen
try:
    vectordb = Chroma(
        persist_directory=str(DB_DIR),
        embedding_function=embedding_model
    )
    print("Databas laddad!")
    # OBS: count() kan vara tungt om indexet är trasigt, men vi testar
    print("Antal chunks:", vectordb._collection.count())
    
except Exception as e:
    print(f"\nKRITISKT FEL VID LADDNING AV DATABAS: {e}")
    print("Detta beror troligen på versionsskillnader mellan Colab och din dator.")

Laddar embedding-modell...
Laddar databas från C:\Users\Dator\Documents\Data_Science\11_Examensarbete\green_power_sweden\data\03_vector_db\green_power_sweden_db...
Databas laddad!
Antal chunks: 164926


### Testa sökning (Utan LLM)

In [2]:
# ==========================================================
# 4. TESTA SÖKNING (utan LLM)
# ==========================================================
query = "Får man bygga solcellsparker på jordbruksmark?"
print(f"\nTestar retrieval för frågan: '{query}'")

docs = vectordb.similarity_search(query, k=3)

print("\n--- RESULTAT (UTAN LLM) ---")
for i, doc in enumerate(docs):
    print(f"\n--- Dokument {i + 1} ---")
    print("Källa:", doc.metadata.get("source", "Okänd"))
    print(doc.page_content[:300], "...")


Testar retrieval för frågan: 'Får man bygga solcellsparker på jordbruksmark?'

--- RESULTAT (UTAN LLM) ---

--- Dokument 1 ---
Källa: Samrådsanmälan_enligt_12_kap_6_MB_Solcellspark_Vedum_justerad_220307(20325894) (0)_TMP.pdf
1. Inledning
1.1. Bakgrund
EEW planerar att anlägga en solcellspark strax nordöst om Vedum, Vara kommun,
Västra Götalands län, Figur 1.
Figur 1. Översikt planerad solcellspark.
Etableringen avses att uppföras på jordbruksmark samt ängs- och betesmark med inslag
av träd och omfattar en inhägnad solce ...

--- Dokument 2 ---
Källa: MMD_2024-05-14_M2122-24_nej solpark jordbruksmark Haninge.pdf
1. Inledning
1.1. Bakgrund
EEW planerar att anlägga en solcellspark strax utanför om Husby, Haninge kommun,
Stockholms län, Figur 1.
Figur 1. Översikt planerad solcellspark. Solparkens projektområden är markerade med blått.
Etableringen avses att uppföras på åkermark och omfattar en inhägnad solcell ...

--- Dokument 3 ---
Källa: Samrådsunderlag_Hylinge_1.0(22895278) (0)_TMP.pd

### Koppla in Gemini (Chatt)

In [11]:
# ==========================================================
# 5. Skapa Gemini LLM
# ==========================================================
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.3
)


# ==========================================================
# 6. Promptmall (ny LCEL-kompatibel)
# ==========================================================
prompt = ChatPromptTemplate.from_template("""
Du är *Solaris Legal*, expert på svensk juridik för solcellsparker.
Besvara frågan baserat på följande kontext.

Om svaret inte finns i kontexten: svara 
"Jag hittar inte svaret i de dokument jag har tillgång till."

Hitta inte på fakta. Hänvisa till källor om möjligt.

KONTEXST:
{context}

FRÅGA:
{question}

SVAR:
""")

retriever = vectordb.as_retriever(search_kwargs={"k": 5})

# ==========================================================
# 7. Skapa RAG-kedja (MED KÄLLHÄNVISNINGAR)
# ==========================================================

# En funktion som bakar ihop text + metadata så LLM:en ser det
def format_docs_with_sources(docs):
    formatted_texts = []
    for doc in docs:
        # Hämta metadata (som vi sparade i notebook 04)
        path = doc.metadata.get("full_path", "Okänd sökväg")
        page = doc.metadata.get("page", "Okänd sida")
        content = doc.page_content
        
        # Skapa en sträng som tydligt visar var texten kommer ifrån
        entry = f"KÄLLA: {path} (Sida {page})\nTEXT: {content}\n----------------"
        formatted_texts.append(entry)
        
    return "\n\n".join(formatted_texts)

def run_rag(question):
    print(f"Letar efter svar på: '{question}'...")
    
    # 1. Hämta dokument (ÖKA k FÖR ATT HITTA MER!)
    # Vi ökar k till 10 eller 15 för att öka chansen att hitta rätt dokument
    retrieved_docs = vectordb.similarity_search(question, k=10)
    
    # Debug: Se vad den faktiskt hittade (syns bara i loggen)
    print(f"Hittade {len(retrieved_docs)} relevanta textsnuttar.")
    
    # 2. Formatera kontexten MED källor
    context_text = format_docs_with_sources(retrieved_docs)
    
    # 3. Skicka till Gemini
    chain = prompt | llm | StrOutputParser()
    response = chain.invoke({"context": context_text, "question": question})
    
    return response

# ==========================================================
# 8. Testa RAG
# ==========================================================
print("\n\n🚀 Testar RAG med Gemini (Nu med källor!):\n")

# Fråga 1
answer = run_rag("Får man bygga solcellsparker på jordbruksmark?")

# Fråga 2 (Kalmar)
# answer = run_rag("Vilken är den största solcellsparken man har ansökt om att få bygga i Kalmar och hur stor effekt är den på?")

print("\nSVAR:")
print(answer)



🚀 Testar RAG med Gemini (Nu med källor!):

Letar efter svar på: 'Får man bygga solcellsparker på jordbruksmark?'...
Hittade 10 relevanta textsnuttar.

SVAR:
Ja, det finns flera exempel i de tillgängliga dokumenten där det planeras att anlägga solcellsparker på jordbruksmark eller åkermark.

Exempel:
*   En etablering av en solcellspark nordöst om Vedum, Vara kommun, avses att uppföras på jordbruksmark samt ängs- och betesmark (KÄLLA: Underlag för kartläggning från länsstyrelser\2019 - maj 2022 - Handlingar 1 jan 2019-31 maj 2022\Västra Götaland (NYA, DAVID)\525-10968-2022\Samrådsanmälan_enligt_12_kap_6_MB_Solcellspark_Vedum_justerad_220307(20325894) (0)_TMP.pdf, Sida 5; Underlag för kartläggning från länsstyrelser\2022 Handlingar 1 juni-31 dec\Västra Götaland\10968-2022 Anmälan för samråd om uppförande av solcellspark i Vara kommun(23197173) (0)_TMP.pdf, Sida 14).
*   En planerad solcellspark norr om Gusum, Valdemarsviks kommun, avses att uppföras på åkermark och/eller skogsmark (KÄL